This is a pseudocode workflow for reproducing the analysis; for a detailed tutorial, please visit
https://starlitnightly.github.io/omicverse/

Bulk RNA-seq process

In [ ]:
#Dump
fasterq-dump filename -e 32 2> filename_dump.log
#QC
fastp -i filename.fastq -o filename.fq.gz -f 2 -t 1 -q 15 -u 20 -n 0 -l 35 -w 16 -j filename_fastp.json -h filename_fastp.html -R filename fastp report
#Mapping
hisat2 -p 55 --dta -x genome_Index -U filename.fq.gz --un-conc-gz filename.fq.gz --new-summary --summary-file filename.align.log | samtools sort -@ 24 -m 6G -o filename.bam
#Assembly
stringtie filename -o filename.gtf -G genomic.gtf
taco_run -v -p 55 --gtf-expr-attr cov --filter-min-expr 2 --filter-splice-juncs -p 55 --ref-genome-fasta genomic_fasta -o TACO GTFs.txt 2> TACO.log
#Transcripts filter
gffcompare -r genomic.gtf assembly.gtf
#Quant
align_and_estimate_abundance.pl --transcripts Transcripts.fasta --seqType fq --gene_trans_map Transcripts.fasta.gene_trans_map --samples_file samples.txt --est_method salmon --thread_count 32 --prep_reference
abundance_estimates_to_matrix.pl --est_method salmon --gene_trans_map Transcripts.fasta.gene_trans_map --out_prefix Transcripts --name_sample_by_basedir Salmon/*/quant.sf 2> matrix.log
#Sequence GO annotation
blastp -query Transcripts.fasta.transdecoder_dir/longest_orfs.pep -db  uniprot_sprot.pep -out blastp.outfmt6 -max_target_seqs 1 -outfmt 6 -evalue 1e-5 -num_threads 32
hmmsearch --cpu 8 --domtblout pfam.domtblout Pfam-A.hmm Transcripts.fasta.transdecoder_dir/longest_orfs.pep 
Trinotate-Trinotate-v4.0.2/util/extract_GO_assignments_from_Trinotate_xls.pl --Trinotate_xls Trinotate.xls --G > go_annotations.txt
emapper.py -i Transcripts.fasta.transdecoder.pep -o output -m diamond --dmnd_ignore_warnings --cpu 55 --sensmode very-sensitive --id 40 --data_dir dir --dmnd_db eggnog_proteins.dmnd --evalue 1e-5
#TF annotation
hmmsearch --cpu 8 --domtblout pfam.domtblout TF.hmm Transcripts.fasta.transdecoder_dir/longest_orfs.pep 
diamond blastx --db /TFDB/TFa.dmnd --query ../NewGTF/Transcripts.fasta -e 1e-5 --outfmt 6 --quiet --very-sensitive --max-target-seqs 1 --id 60 --out TF_predict.txt

In [ ]:
#WGCNA in R
library(WGCNA)

setwd("")
femData<- read.table("Transcripts.gene.TPM.not_cross_norm", header = TRUE, row.names = 1)
femData <- femData[!grepl("^gene-Trna", rownames(femData)), ]
femData= femData[rowSums(femData>=5) >=2, ] 
femData <- log2(femData + 1)
var_values <- apply(femData, 1, var)
cutoff <- quantile(var_values, 0.25)
femData <- femData[var_values >= cutoff, ]
datExpr0 = as.data.frame(t(femData)) 
powers = c(c(1:10), seq(from = 12, to=20, by=2))
sft = pickSoftThreshold(datExpr0, powerVector = powers, verbose = 5)
sizeGrWindow(9, 5)
par(mfrow = c(1,2));
cex1 = 1;
plot(sft$fitIndices[,1], -sign(sft$fitIndices[,3])*sft$fitIndices[,2],
     xlab="Soft Threshold (power)",ylab="Scale Free Topology Model Fit,signed R^2",type="n",
     main = paste("Scale independence"));
text(sft$fitIndices[,1], -sign(sft$fitIndices[,3])*sft$fitIndices[,2],
     labels=powers,cex=cex1,col="red");
abline(h=0.8,col="red") 
plot(sft$fitIndices[,1], sft$fitIndices[,5],
     xlab="Soft Threshold (power)",ylab="Mean Connectivity", type="n",
     main = paste("Mean connectivity"))
text(sft$fitIndices[,1], sft$fitIndices[,5], labels=powers, cex=cex1,col="red") 

datExpr0[] <- lapply( datExpr0, as.numeric)
WGCNA_matrix<-datExpr0
enableWGCNAThreads()
#
TOM <- TOMsimilarityFromExpr(WGCNA_matrix,power = 8,
                             networkType = "unsigned",
                             nThreads = 50)
dissTOM <- 1-TOM
geneTree <- hclust(as.dist(dissTOM),method = "average")
minModuleSize <-100# 
dynamicMods<- cutreeDynamic(
  dendro= geneTree,
  distM= dissTOM,
  deepSplit= 4,
  pamStage= TRUE,
  pamRespectsDendro= FALSE,
  minClusterSize= minModuleSize)
dynamicColors =labels2colors(dynamicMods)
p1<-plotDendroAndColors(
  geneTree,
  dynamicColors,
  "DynamicTree Cut",
  dendroLabels= FALSE, hang = 0.03,
  addGuide= TRUE,
  guideHang= 0.05,
  main= "Gene dendrogram and module colors")
print(p1)
dev.off()

MEList<- moduleEigengenes(WGCNA_matrix,colors = dynamicColors)
MEs<- MEList$eigengenes
MEDiss<- 1-cor(MEs)
METree<- hclust(as.dist(MEDiss), method = "average")
p1<-plot(METree,
         main= "Clustering of module eigengenes",
         xlab= "", sub = "")
abline(h = 0.2, col = "red", lty = 2)
print(p1)
dev.off()

MEDissThres<- 0.2
merge<- mergeCloseModules(WGCNA_matrix,dynamicColors,cutHeight= MEDissThres,verbose = 3)
mergedColors <- merge$colors
table(mergedColors)
mergedMEs<- merge$newMEs
p1<-plotDendroAndColors(
  geneTree,
  cbind(dynamicColors,mergedColors),
  c("DynamicTree Cut", "Merged dynamic"),
  dendroLabels= FALSE,
  hang= 0.03,
  addGuide= TRUE,
  guideHang= 0.05)
print(p1)
dev.off()

table(mergedColors)
write.table(table(mergedColors), file = "Mergedcolors.txt", sep = "\t")

MEDiss<- 1-cor(mergedMEs)
METree <- hclust(as.dist(MEDiss),method = "average")
par(mar = c(1, 4, 3, 2))
p1<-plot(METree,
         main= "Clustering of module eigengenes",
         xlab= "", sub = "")
print(p1)

datME=moduleEigengenes(WGCNA_matrix,mergedColors)[[1]]
color1=as.character(mergedColors)
datKME=signedKME(WGCNA_matrix, datME)
dataExpr1=as.data.frame(t(WGCNA_matrix))
datSummary=rownames(dataExpr1)
datout=data.frame(datSummary,colorNEW=color1,datKME )
write.table(datout, "kMEs.xls", sep="\t", row.names=F,quote=F)

dir.create('cytoscape')
allModules = unique(mergedColors)
probes= colnames(WGCNA_matrix)
for (module in allModules) {
  inModule = is.finite(match(mergedColors, module))
  modProbes = probes[inModule]
  modTOM = TOM[inModule, inModule]
  dimnames(modTOM) = list(modProbes, modProbes)
  cyt = exportNetworkToCytoscape(modTOM,
                                 edgeFile = paste("cytoscape/CytoscapeInput-edges-", module, ".txt", sep=""),
                                 nodeFile = paste("cytoscape/CytoscapeInput-nodes-", module, ".txt", sep=""),
                                 weighted = TRUE,threshold = 0.02,nodeNames = modProbes,nodeAttr = mergedColors[inModule])
}

Alldegrees <- intramodularConnectivity(TOM, mergedColors)
Alldegrees <- data.frame(
  Gene = colnames(WGCNA_matrix), 
  Module = mergedColors,
  Alldegrees
)
write.table(Alldegrees,"connetivity.xls", sep="\t", row.names=T,quote=F)

library(reshape2)
samples <- read.table("./samples1.txt", header = FALSE, stringsAsFactors = FALSE)
df <- dcast(samples, V2 ~ V1, value.var = "V1", fill = 0)
rownames(df) <- df$V2
df <- df[, -1] 
df[is.na(df)] <- 0
df[ df !=0] <- 1
print(df)

traitData = df
allTraits = traitData

traitSamples =rownames(allTraits)
expSamples = rownames(datExpr0)
traitRows = match(expSamples, traitSamples)
datTraits = allTraits[traitRows, ]
rownames(datTraits) 
nSamples = nrow(WGCNA_matrix);
moduleTraitCor = cor(mergedMEs, datTraits, use = "p");
moduleTraitPvalue = corPvalueStudent(moduleTraitCor, nSamples);

ScRNA-seq precess

In [ ]:
cellranger mkref --genome=genome --fasta=genome_asta --genes=genomic.gtf
cellranger count --id=genome --transcriptome=reference --fastqs=fastq_files --sample=sample --localcores=24 --localmem=256

In [ ]:
import omicverse as ov
print(f"omiverse version: {ov.__version__}")
import scanpy as sc
print(f"scanpy version: {sc.__version__}")
ov.ov_plot_set()

In [ ]:
#Merge data
adata11 = sc.read_10x_h5(filename="blood/SRR18899520/wssvblood/outs/filtered_feature_bc_matrix.h5")
adata11.obs['tissue']='Blood' 
adata11.obs['project']='p1' 
adata11.obs['batch']='batch1' 
adata11.obs['condiction']='wssv'
adata1 = sc.read_10x_h5(filename="blood/SRR18899521/bloodout/outs/filtered_feature_bc_matrix.h5")
adata1.obs['tissue']='Blood' 
adata1.obs['project']='p1' 
adata1.obs['batch']='batch1' 
adata1.obs['condiction']='normal'
...
adata=sc.concat([adata1,...adata6,adata7,adata8,adata9,adata10,adata11],join='outer')

In [ ]:
#QC
adata.var_names_make_unique()
adata.obs_names_make_unique()
adata.var["mt"] = adata.var_names.str.startswith("KEG")
from scipy.sparse import issparse
if issparse(adata.X):
    adata.obs['nUMIs'] = adata.X.toarray().sum(axis=1)
    adata.obs['mito_perc'] = adata[:, adata.var["mt"]].X.toarray().sum(axis=1) / adata.obs['nUMIs'].values
    adata.obs['detected_genes'] = (adata.X.toarray() > 0).sum(axis=1)
else:
    adata.obs['nUMIs'] = adata.X.sum(axis=1)
    adata.obs['mito_perc'] = adata[:, adata.var["mt"]].X.sum(axis=1) / adata.obs['nUMIs'].values
    adata.obs['detected_genes'] = (adata.X > 0).sum(axis=1)
adata = adata[adata.obs['mito_perc'] <= 0.2]
qc_thresholds = {'mito_perc': 0.2, 'nUMIs': 500, 'detected_genes': 250}
adata = ov.pp.qc(adata, tresh=qc_thresholds,batch_key='batch')

Annotation with known marker

In [ ]:
#Hemocytes annotation
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
fig, axes = plt.subplots(3, 3, figsize=(18, 18))
cmap = 'Reds'
ov.utils.embedding(adata, basis='X_umap', color='LOC113823934', show=False, ax=axes[0, 0], cmap=cmap)#TGase1 
ov.utils.embedding(adata, basis='X_umap', color='LOC113823945', show=False, ax=axes[0, 1], cmap=cmap)#TGase
ov.utils.embedding(adata, basis='X_umap', color='LOC113814237', show=False, ax=axes[0, 2], cmap=cmap)#Notch1
ov.utils.embedding(adata, basis='X_umap', color='LOC113804553', show=False, ax=axes[1, 0], cmap=cmap)#CTL2 
ov.utils.embedding(adata, basis='X_umap', color='LOC113809352', show=False, ax=axes[1, 1], cmap=cmap)#SVC5
ov.utils.embedding(adata, basis='X_umap', color='LOC113822052', show=False, ax=axes[1, 2], cmap=cmap)#Ast2
ov.utils.embedding(adata, basis='X_umap', color='LOC113802754', show=False, ax=axes[2, 0], cmap=cmap)#CHF
ov.utils.embedding(adata, basis='X_umap', color='LOC113816267', show=False, ax=axes[2, 1], cmap=cmap)#CrustinL
ov.utils.embedding(adata, basis='X_umap', color='LOC113801825', show=False, ax=axes[2, 2], cmap=cmap)#Crustin

In [ ]:
#Macrophage-like hemocytes subpopulation 
'''
CHIT1 LOC113822241
Lyz1 LOC113802295
LIPF LOC113823443
LGMN LOC113827868
Nlrp3 LOC113828560
NAGA LOC113810366
zfh1 LOC113824903
Casp1 LOC113800234
NPC2 LOC113806913
'''
fig, axes = plt.subplots(3, 3, figsize=(18, 18))
cmap = 'Purples'
ov.utils.embedding(adat2, basis='X_umap', color='LOC113822241', show=False, ax=axes[0, 0], cmap=cmap) 
ov.utils.embedding(adat2, basis='X_umap', color='LOC113802295', show=False, ax=axes[0, 1], cmap=cmap)
ov.utils.embedding(adat2, basis='X_umap', color='LOC113823443', show=False, ax=axes[0, 2], cmap=cmap)
ov.utils.embedding(adat2, basis='X_umap', color='LOC113827868', show=False, ax=axes[1, 0], cmap=cmap)
ov.utils.embedding(adat2, basis='X_umap', color='LOC113828560', show=False, ax=axes[1, 1], cmap=cmap)
ov.utils.embedding(adat2, basis='X_umap', color='LOC113810366', show=False, ax=axes[1, 2], cmap=cmap)
ov.utils.embedding(adat2, basis='X_umap', color='LOC113824903', show=False, ax=axes[2, 0], cmap=cmap)
ov.utils.embedding(adat2, basis='X_umap', color='LOC113800234', show=False, ax=axes[2, 1], cmap=cmap)
ov.utils.embedding(adat2, basis='X_umap', color='LOC113806913', show=False, ax=axes[2, 2], cmap=cmap)

In [ ]:
#Hep
#E cell
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
cmap = 'Blues'
ov.utils.embedding(adata, basis='X_umap', color='LOC113819379', show=False, ax=axes[0, 0], cmap=cmap)
ov.utils.embedding(adata, basis='X_umap', color='LOC113823098', show=False, ax=axes[0, 1], cmap=cmap)
ov.utils.embedding(adata, basis='X_umap', color='LOC113819117', show=False, ax=axes[0, 2], cmap=cmap)
ov.utils.embedding(adata, basis='X_umap', color='LOC113823089', show=False, ax=axes[1, 0], cmap=cmap)
ov.utils.embedding(adata, basis='X_umap', color='LOC113827499', show=False, ax=axes[1, 1], cmap=cmap)
#B cell
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
cmap = 'Blues'
ov.utils.embedding(adata, basis='X_umap', color='LOC113823145', show=False, ax=axes[0, 0], cmap=cmap)
ov.utils.embedding(adata, basis='X_umap', color='LOC113809598', show=False, ax=axes[0, 1], cmap=cmap)
ov.utils.embedding(adata, basis='X_umap', color='LOC113822815', show=False, ax=axes[0, 2], cmap=cmap)
ov.utils.embedding(adata, basis='X_umap', color='LOC113810775', show=False, ax=axes[1, 0], cmap=cmap)
ov.utils.embedding(adata, basis='X_umap', color='LOC113804689', show=False, ax=axes[1, 1], cmap=cmap)
#F cell 
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
ov.utils.embedding(adata, basis='X_umap', color='LOC113819352', show=False, ax=axes[0, 0], cmap=cmap)
ov.utils.embedding(adata, basis='X_umap', color='LOC113817198', show=False, ax=axes[0, 1], cmap=cmap)
ov.utils.embedding(adata, basis='X_umap', color='LOC113811244', show=False, ax=axes[0, 2], cmap=cmap)
ov.utils.embedding(adata, basis='X_umap', color='LOC113814344', show=False, ax=axes[1, 0], cmap=cmap)
ov.utils.embedding(adata, basis='X_umap', color='LOC113819139', show=False, ax=axes[1, 1], cmap=cmap)
#F cell 
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
ov.utils.embedding(adata, basis='X_umap', color='LOC113809896', show=False, ax=axes[0, 0], cmap=cmap)
ov.utils.embedding(adata, basis='X_umap', color='LOC113820201', show=False, ax=axes[0, 1], cmap=cmap)
ov.utils.embedding(adata, basis='X_umap', color='LOC113819222', show=False, ax=axes[0, 2], cmap=cmap)
ov.utils.embedding(adata, basis='X_umap', color='LOC113806454', show=False, ax=axes[1, 0], cmap=cmap)
ov.utils.embedding(adata, basis='X_umap', color='LOC113813404', show=False, ax=axes[1, 1], cmap=cmap)
#R cell
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
ov.utils.embedding(adata, basis='X_umap', color='LOC113817261', show=False, ax=axes[0, 0], cmap=cmap)
ov.utils.embedding(adata, basis='X_umap', color='LOC113810053', show=False, ax=axes[0, 1], cmap=cmap)
ov.utils.embedding(adata, basis='X_umap', color='LOC113799989', show=False, ax=axes[0, 2], cmap=cmap)
ov.utils.embedding(adata, basis='X_umap', color='LOC113817260', show=False, ax=axes[1, 0], cmap=cmap)
ov.utils.embedding(adata, basis='X_umap', color='LOC113810057', show=False, ax=axes[1, 1], cmap=cmap)
genes_of_interest = [#E
 
'LOC113819379',
'LOC113823098',
'LOC113819117',
'LOC113823089',
'LOC113827499',
    #R
    'LOC113817261', 
'LOC113810053',
'LOC113799989',
'LOC113817260',
'LOC113810057', 
    #F
  'LOC113809896',
'LOC113820201',
'LOC113819222',
'LOC113806454',
'LOC113813404',
'LOC113819352',
'LOC113811244',
'LOC113814344',
'LOC113819139',
  #B
'LOC113823145',
'LOC113809598',
'LOC113822815',
'LOC113810775',
'LOC113804689',
#CTL
    'LOC113804553',
'LOC113809352',
'LOC113816267', 
'LOC113801825', 
]
sc.pl.dotplot(
    adata,
    var_names=genes_of_interest, 
    groupby='leiden_res', 
    cmap='Blues',
    standard_scale='var'
)

Novel cell type annotaion

In [ ]:
##DEG
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.tl.rank_genes_groups(
    adata, groupby="major_celltype", use_raw=False,
    method="t-test",)
groups = adata.uns['rank_genes_groups']['names'].dtype.names
merged_df = pd.DataFrame()
for group in groups:
    ranked_genes_df = sc.get.rank_genes_groups_df(adata, group=group)
    ranked_genes_df = ranked_genes_df[['names', 'scores']]
    ranked_genes_df.rename(columns={'scores': f'{group}_score'}, inplace=True)
    if merged_df.empty:
        merged_df = ranked_genes_df
    else:
        merged_df = pd.merge(merged_df, ranked_genes_df, on='names', how='outer')
merged_df.to_csv ('ranksocre.txt',sep='\t')
groups = adata.uns['rank_genes_groups']['names'].dtype.names
merged_df = pd.DataFrame()
for group in groups:
    ranked_genes_df = sc.get.rank_genes_groups_df(adata, group=group)
    ranked_genes_df = ranked_genes_df[['names', 'logfoldchanges']]
    ranked_genes_df.rename(columns={'logfoldchanges': f'{group}_lgFC'}, inplace=True)
    if merged_df.empty:
        merged_df = ranked_genes_df
    else:
        merged_df = pd.merge(merged_df, ranked_genes_df, on='names', how='outer')
merged_df.to_csv ('rankFC.txt',sep='\t')

In [ ]:
#AUCell for GO
pathway_dict=ov.utils.geneset_prepare('output.gmt')
adata_aucs=ov.single.pathway_aucell_enrichment(adata,pathways_dict=pathway_dict, num_workers=48)
adata_aucs.obs=adata[adata_aucs.obs.index].obs
adata_aucs.obsm=adata[adata_aucs.obs.index].obsm
adata_aucs.obsp=adata[adata_aucs.obs.index].obsp
sc.tl.dendrogram(adata_aucs,'major_celltype',use_rep="X_umap")
sc.tl.rank_genes_groups(adata_aucs, 'major_celltype', method='t-test',n_genes=100)
sc.tl.rank_genes_groups(adata_aucs, 'major_celltype', method='t-test')
import pandas as pd
adata=adata_aucs
groups = adata.uns['rank_genes_groups']['names'].dtype.names
merged_df = pd.DataFrame()
for group in groups:
    ranked_genes_df = sc.get.rank_genes_groups_df(adata, group=group)
    ranked_genes_df = ranked_genes_df[['names', 'scores']]
    ranked_genes_df.rename(columns={'scores': f'{group}_score'}, inplace=True)
    if merged_df.empty:
        merged_df = ranked_genes_df
    else:
        merged_df = pd.merge(merged_df, ranked_genes_df, on='names', how='outer')
merged_df.to_csv ('subranksocre.txt',sep='\t')
groups = adata.uns['rank_genes_groups']['names'].dtype.names
merged_df = pd.DataFrame()
for group in groups:
    ranked_genes_df = sc.get.rank_genes_groups_df(adata, group=group)
    ranked_genes_df = ranked_genes_df[['names', 'logfoldchanges']]
    ranked_genes_df.rename(columns={'logfoldchanges': f'{group}_lgFC'}, inplace=True)
    if merged_df.empty:
        merged_df = ranked_genes_df
    else:
        merged_df = pd.merge(merged_df, ranked_genes_df, on='names', how='outer')
merged_df.to_csv ('subrankFC.txt',sep='\t')